# MIMIC Notes and Structured Data Prep

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pdb
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import pickle
from tqdm import tqdm_notebook as tqdm
from ast import literal_eval
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

In [3]:
path = Path('data')
workdir = path/'workdir'
figdir = workdir/'figures'

## Functions

In [4]:
def change_name(col_name):
  if '(' not in col_name:
    return col_name
  cols = literal_eval(col_name)
  return f'{cols[0]}_{cols[1]}'

In [5]:
def data_interval(x):
  if pd.isnull(x):
    return -1
  
  if 0 < x <= 1:
    return 0
  elif 1 < x <= 2:
    return 1
  elif 2 < x <= 3:
    return 2
  elif 3 < x <= 4:
    return 3
  elif 4 < x <= 5:
    return 4
  elif 5 < x <= 6:
    return 5
  elif 6 < x <= 7:
    return 6
  elif 7 < x <= 8:
    return 7
  elif 8 < x <= 9:
    return 8
  elif 9 < x <= 10:
    return 9
  elif 10 < x <= 11:
    return 10
  elif 11 < x <= 12:
    return 11
  elif 12 < x <= 13:
    return 12
  elif 13 < x <= 14:
    return 13
  elif 14 < x <= 15:
    return 14
  else:
    return 15

def icu_adm_label(x):
  if 0 <= x <= 1:
    return -1 # unused notes due to data leakage
  elif 1 < x <= 3:
    return 1 # imminent ICU admission
  elif 3 < x <= 5:
    return -1 # unused notes due to data leakage
  else:
    return 0 # delayed ICU admission

## Processing 

In [ ]:
notes_df = pd.read_csv(path/'unstructured_raw.csv', parse_dates=['intime', 'admittime', 'charttime'])
notes_df.drop_duplicates(inplace=True)

vitals_df = pd.read_csv(path/'structured_raw_with_meta.csv', parse_dates=['charttime'])
vitals_df.drop_duplicates(inplace=True)

In [ ]:
notes_hadms = notes_df['hadm_id'].unique()
vitals_hadms = vitals_df['hadm_id'].unique()

# Extract common `hadm_id` and filter out those that do not appear in both dfs
common_hadms = set(vitals_df['hadm_id'].unique()).intersection(notes_df['hadm_id'].unique())

print(f"Number of encounters that definitely have structured vitals data: {len(vitals_hadms)}")
print(f"Number of encounters that definitely have clinical notes: {len(notes_hadms)}")
print(f"Number of encounters that have both vitals and clinical notes: {len(common_hadms)}")

In [ ]:
notes_common = notes_df[notes_df['hadm_id'].isin(common_hadms)].reset_index(drop=True)
vitals_common = vitals_df[vitals_df['hadm_id'].isin(common_hadms)].reset_index(drop=True)

# sanity check
s, n = set(vitals_common['hadm_id'].unique()), set(notes_common['hadm_id'].unique())
assert(s.symmetric_difference(n) == set())

vitals_common.shape, notes_common.shape

In [ ]:
notes_common['note'] = notes_common['category'].str.cat(notes_common['description'], sep='\n')
notes_common['note'] = notes_common['note'].str.cat(notes_common['text'], sep='\n')
notes_common.drop(columns=['category', 'description', 'text'], inplace=True) 

notes_common = pd.DataFrame(notes_common.groupby(['hadm_id', 'intime', 'admittime', 'charttime'])['note'].apply('\n'.join)).reset_index()
notes_common['category'] = notes_common['note'].apply(lambda x: x.split('\n')[0])
notes_common.shape

In [ ]:
# Remove redundant info by filling in each time column with the value of the var
vitals_common = vitals_common.groupby(['hadm_id','charttime','intime','admittime']).sum(min_count = 1).reset_index()
# Groupby ffill 
vitals_common = vitals_common.groupby(['hadm_id'], as_index=False).apply(lambda group: group.ffill())
# Groupby bfill 
vitals_common = vitals_common.groupby(['hadm_id'], as_index=False).apply(lambda group: group.bfill())
vitals_common = vitals_common.fillna(vitals_common.median())
vitals_common.shape

In [ ]:
notes_common.to_csv(path/'unstructured_notes_proc.csv', index=False)
vitals_common.to_csv(path/'structured_vitals_proc.csv', index=False)

## Compute Statistics Dev

In [ ]:
vitals_common = pd.read_csv(path/'structured_vitals_proc.csv', parse_dates=['charttime'])
vitals_common.drop(['intime', 'admittime'], inplace=True, axis=1)
vitals_common.shape

In [ ]:
x = pd.DataFrame(vitals_common.groupby('hadm_id').size(), columns=['size']).reset_index()
hadms = x.loc[(x['size'] >= 10) & (x['size'] <= 20)].sample(5)['hadm_id'].tolist()
x.loc[x['hadm_id'].isin(hadms)]

In [ ]:
dev_subset = vitals_common.loc[(vitals_common['hadm_id'].isin(hadms))].reset_index(drop=True)
print(dev_subset.shape)
print(dev_subset.columns)

In [ ]:
var_cols = dev_subset.columns[2:]
print(len(var_cols))
running_stats = ['min', 'mean', 'median', 'std', 'max']
dfs = []

In [ ]:
for hadm_id, group_df in tqdm(dev_subset.groupby('hadm_id'), desc='Encounters'):
  df = group_df.copy()
  var_df = df[var_cols].reset_index(drop=True) # save the original vals for later
  
  df.set_index('charttime', inplace=True) # set charttime as index for rolling 24h
  stats_df = df[var_cols].rolling('24h').agg(running_stats)
  
  df = pd.DataFrame(stats_df.to_records()) # flatten the resulting dataframe
  df.insert(loc=1, column='hadm_id', value=hadm_id)
  
  df.rename(columns=change_name, inplace=True) # rename columns
  df = pd.concat([df, var_df], axis=1) # add the original vals back
  
  # reorder vars such that the columns are var, var_stat...
  stats_cols = df.columns[2:]
  all_cols = []
  for var in var_cols:
    all_cols.append(var)
    for stat in stats_cols:
      if f'{var}_' in stat:
        all_cols.append(stat)
        
  order = list(df.columns[:2]) + all_cols
  df = df[order]
  dfs.append(df)

dev_subset_stats = pd.concat(dfs)
dev_subset_stats.reset_index(drop=True, inplace=True)
dev_subset_stats['charttime'] = pd.to_datetime(dev_subset_stats['charttime'])

std_cols = [col for col in dev_subset_stats.columns if 'std' in col]
dev_subset_stats[std_cols] = dev_subset_stats[std_cols].fillna(0)

dev_subset_stats = dev_subset_stats[['hadm_id', 'charttime'] + list(dev_subset_stats.columns[2:])]

In [ ]:
print(dev_subset_stats.shape)
dev_subset_stats.columns

## Prep data for model

### Merge

In [ ]:
notes_common = pd.read_csv(path/'unstructured_notes_proc.csv', parse_dates=['intime', 'admittime', 'charttime'])
notes_common.drop(columns=['category'], inplace=True)

vitals_common_solo = pd.read_csv(path/'structured_vitals_proc.csv', parse_dates=['intime', 'admittime', 'charttime'])
vitals_common_stats = pd.read_csv(path/'structured_vitals_stats.csv', parse_dates=['charttime'])

# pickle.dump(list(vitals_common_solo.columns[2:]), open(path/'vitals_solo_cols.pkl', 'wb'))
# pickle.dump(list(vitals_common_stats.columns[2:]), open(path/'vitals_stats_cols.pkl', 'wb'))

print(vitals_common_solo.shape, vitals_common_stats.shape, notes_common.shape, vitals_common_solo['hadm_id'].nunique(), vitals_common_stats['hadm_id'].nunique(), notes_common['hadm_id'].nunique())

#### Merge Dev

In [ ]:
main = ['hadm_id', 'charttime']
sub1 = ['hr', 'hr_max', 'temp', 'temp_min', 'glucose', 'glucose_std', 'map', 'map_median']
sub2 = ['admittime', 'intime', 'note']

In [ ]:
x = pd.DataFrame(notes_common.groupby('hadm_id').size(), columns=['size']).reset_index()
hadms = x.loc[(x['size'] >= 2) & (x['size'] <= 15)].sample(5)['hadm_id'].tolist()

subset_stats = vitals_common_stats.loc[(vitals_common_stats['hadm_id'].isin(hadms))][main + sub1].copy().reset_index(drop=True)

subset_notes = notes_common.loc[(notes_common['hadm_id'].isin(hadms))][main + sub2].copy().reset_index(drop=True)
subset_stats.shape, subset_stats['hadm_id'].nunique(), subset_notes.shape, subset_notes['hadm_id'].nunique()

In [ ]:
pd.DataFrame(subset_stats.groupby('hadm_id').size(), columns=['size']).reset_index()

In [ ]:
pd.DataFrame(subset_notes.groupby('hadm_id').size(), columns=['size']).reset_index()

In [ ]:
subset_stats.sort_values(by='charttime', inplace=True)
subset_stats.reset_index(inplace=True, drop=True)

subset_notes.sort_values(by='charttime', inplace=True)
subset_notes.reset_index(inplace=True, drop=True)

df = pd.merge_asof(subset_notes, subset_stats, left_on='charttime', right_on='charttime', by='hadm_id')

cols = ['hr', 'hr_max', 'temp', 'temp_min', 'glucose', 'glucose_std', 'map', 'map_median']

df = df.groupby(['hadm_id'], as_index=False).apply(lambda group: group.bfill())
df[cols] = df[cols].fillna(df[cols].median())

In [ ]:
i = -1

In [ ]:
i += 1
print(hadms[i])

In [ ]:
subset_stats[subset_stats['hadm_id'] == hadms[i]].reset_index(drop=True)

In [ ]:
subset_notes[subset_notes['hadm_id'] == hadms[i]].reset_index(drop=True)

In [ ]:
df[df['hadm_id'] == hadms[i]].reset_index(drop=True)

In [ ]:
df.shape

#### Final Merge

In [ ]:
vitals_common_stats.sort_values(by='charttime', inplace=True)
vitals_common_stats.reset_index(inplace=True, drop=True)

notes_common.sort_values(by='charttime', inplace=True)
notes_common.reset_index(inplace=True, drop=True)

mm_notes_vitals = pd.merge_asof(notes_common, vitals_common_stats, left_on='charttime', right_on='charttime', by='hadm_id')

str_cols = pickle.load(open(path/'vitals_stats_cols.pkl', 'rb'))

mm_notes_vitals = mm_notes_vitals.groupby(['hadm_id'], as_index=False).apply(lambda group: group.bfill())
mm_notes_vitals[str_cols] = mm_notes_vitals[str_cols].fillna(mm_notes_vitals[str_cols].median())

In [ ]:
x = pd.DataFrame(mm_notes_vitals.isna().sum(), columns=['sum']).reset_index()
assert(x['sum'].sum() == 0)

In [ ]:
mm_notes_vitals.to_csv(path/'mm_notes_vitals_proc.csv', index=False)

### Labeling

In [6]:
notes_common = pd.read_csv(path/'unstructured_notes_proc.csv', parse_dates=['intime', 'admittime', 'charttime'])
notes_common.drop(columns=['category'], inplace=True)

vitals_common_solo = pd.read_csv(path/'structured_vitals_proc.csv', parse_dates=['intime', 'admittime', 'charttime'])
mm_notes_vitals = pd.read_csv(path/'mm_notes_vitals_proc.csv', parse_dates=['intime', 'admittime', 'charttime'])

drop_cols = ['charttime', 'intime', 'admittime', 'admit_to_icu', 'chart_to_icu', 'interval']
print(vitals_common_solo.shape, notes_common.shape, mm_notes_vitals.shape)

(270288, 12) (53270, 5) (53270, 53)


In [7]:
print(vitals_common_solo.columns)
print(notes_common.columns)
print(mm_notes_vitals.columns)

Index(['hadm_id', 'charttime', 'intime', 'admittime', 'hr', 'sbp', 'dbp',
       'map', 'resp', 'temp', 'spo2', 'glucose'],
      dtype='object')
Index(['hadm_id', 'intime', 'admittime', 'charttime', 'note'], dtype='object')
Index(['hadm_id', 'intime', 'admittime', 'charttime', 'note', 'hr', 'hr_min',
       'hr_mean', 'hr_median', 'hr_std', 'hr_max', 'sbp', 'sbp_min',
       'sbp_mean', 'sbp_median', 'sbp_std', 'sbp_max', 'dbp', 'dbp_min',
       'dbp_mean', 'dbp_median', 'dbp_std', 'dbp_max', 'map', 'map_min',
       'map_mean', 'map_median', 'map_std', 'map_max', 'resp', 'resp_min',
       'resp_mean', 'resp_median', 'resp_std', 'resp_max', 'temp', 'temp_min',
       'temp_mean', 'temp_median', 'temp_std', 'temp_max', 'spo2', 'spo2_min',
       'spo2_mean', 'spo2_median', 'spo2_std', 'spo2_max', 'glucose',
       'glucose_min', 'glucose_mean', 'glucose_median', 'glucose_std',
       'glucose_max'],
      dtype='object')


In [8]:
notes_common['admit_to_icu'] = (notes_common['intime'] - notes_common['admittime'])/np.timedelta64(1, 'D')
notes_common['chart_to_icu'] = (notes_common['intime'] - notes_common['charttime'])/np.timedelta64(1, 'D')

notes_common['interval'] = notes_common['chart_to_icu'].apply(data_interval)
notes_common['imi_adm_label'] = notes_common['interval'].apply(icu_adm_label)
notes_common.shape, notes_common['hadm_id'].nunique()

((53270, 9), 8254)

In [9]:
vitals_common_solo['admit_to_icu'] = (vitals_common_solo['intime'] - vitals_common_solo['admittime'])/np.timedelta64(1, 'D')
vitals_common_solo['chart_to_icu'] = (vitals_common_solo['intime'] - vitals_common_solo['charttime'])/np.timedelta64(1, 'D')

vitals_common_solo['interval'] = vitals_common_solo['chart_to_icu'].apply(data_interval)
vitals_common_solo['imi_adm_label'] = vitals_common_solo['interval'].apply(icu_adm_label)
vitals_common_solo.shape, vitals_common_solo['hadm_id'].nunique()

((270288, 16), 8254)

In [10]:
mm_notes_vitals['admit_to_icu'] = (mm_notes_vitals['intime'] - mm_notes_vitals['admittime'])/np.timedelta64(1, 'D')
mm_notes_vitals['chart_to_icu'] = (mm_notes_vitals['intime'] - mm_notes_vitals['charttime'])/np.timedelta64(1, 'D')

mm_notes_vitals['interval'] = mm_notes_vitals['chart_to_icu'].apply(data_interval)
mm_notes_vitals['imi_adm_label'] = mm_notes_vitals['interval'].apply(icu_adm_label)
mm_notes_vitals.shape, mm_notes_vitals['hadm_id'].nunique()

((53270, 57), 8254)

In [14]:
print(notes_common.shape, notes_common['hadm_id'].nunique())
print(vitals_common_solo.shape, vitals_common_solo['hadm_id'].nunique())
print(mm_notes_vitals.shape, mm_notes_vitals['hadm_id'].nunique())

(53270, 3) 8254
(270288, 10) 8254
(53270, 51) 8254


In [11]:
notes_common.drop(drop_cols, axis=1, inplace=True)
vitals_common_solo.drop(drop_cols, axis=1, inplace=True)
mm_notes_vitals.drop(drop_cols, axis=1, inplace=True)

In [12]:
print(vitals_common_solo.columns)
print(notes_common.columns)
print(mm_notes_vitals.columns)

Index(['hadm_id', 'hr', 'sbp', 'dbp', 'map', 'resp', 'temp', 'spo2', 'glucose',
       'imi_adm_label'],
      dtype='object')
Index(['hadm_id', 'note', 'imi_adm_label'], dtype='object')
Index(['hadm_id', 'note', 'hr', 'hr_min', 'hr_mean', 'hr_median', 'hr_std',
       'hr_max', 'sbp', 'sbp_min', 'sbp_mean', 'sbp_median', 'sbp_std',
       'sbp_max', 'dbp', 'dbp_min', 'dbp_mean', 'dbp_median', 'dbp_std',
       'dbp_max', 'map', 'map_min', 'map_mean', 'map_median', 'map_std',
       'map_max', 'resp', 'resp_min', 'resp_mean', 'resp_median', 'resp_std',
       'resp_max', 'temp', 'temp_min', 'temp_mean', 'temp_median', 'temp_std',
       'temp_max', 'spo2', 'spo2_min', 'spo2_mean', 'spo2_median', 'spo2_std',
       'spo2_max', 'glucose', 'glucose_min', 'glucose_mean', 'glucose_median',
       'glucose_std', 'glucose_max', 'imi_adm_label'],
      dtype='object')


In [13]:
vitals_common_solo.to_csv(path/'modelready_structured.csv', index=False)
notes_common.to_csv(path/'modelready_unstructured.csv', index=False)
mm_notes_vitals.to_csv(path/'modelready_mm.csv', index=False)

## Data Exploration

### Cohort: **notes_all**

Read in all **notes_all** and subset it to get all the data with label not equal to -1 (only data used for modeling). Then get the unique ``hadm_id``'s within that.

In [ ]:
notes_df = pd.read_csv(path/'notes_all_proc.csv', parse_dates=['intime', 'admittime', 'ne_charttime'])
model_notes_df = notes_df[notes_df['imi_adm_label'] != -1].reset_index(drop=True)
hadms = model_notes_df['hadm_id'].unique()

Subset the **notes_cohort** to get details of only those encountered that are used for modeling

In [ ]:
notes_cohort = pd.read_csv(path/'notes_all_cohort.csv')
notes_cohort = notes_cohort[notes_cohort['hadm_id'].isin(hadms)].reset_index(drop=True)

In [ ]:
def group_eth(eth):
  eth = eth.lower()
  if 'white' in eth:
    return 'white'
  elif 'black' in eth:
    return 'black'
  elif 'hispanic' in eth:
    return 'hispanic'
  elif 'asian' in eth:
    return 'asian'
  else:
    return 'other'

notes_cohort['ethnicity'] = notes_cohort['ethnicity'].apply(group_eth)
notes_cohort.loc[notes_cohort['admission_age'] > 100, 'admission_age'] = 100

In [ ]:
print(f"Number of patients in notes cohort: {notes_cohort['subject_id'].nunique()}")

In [ ]:
g = notes_cohort.groupby('expire_flag')['subject_id'].nunique().to_numpy()
print(f"Mortality in notes cohort: {g[1]} ({(g[1]/g.sum())*100:0.1f}%)")

In [ ]:
g = notes_cohort.groupby('gender')['subject_id'].nunique().to_numpy()
print(f"Males in notes cohort: {g[1]} ({(g[1]/g.sum())*100:0.1f}%)")

In [ ]:
print(f"Mean:{notes_cohort.groupby('subject_id')['admission_age'].first().mean():0.1f}")
print(f"STD:{notes_cohort.groupby('subject_id')['admission_age'].first().std():0.1f}")
print(f"25th percentile:{notes_cohort.groupby('subject_id')['admission_age'].first().quantile(0.25):0.1f}")
print(f"75th percentile:{notes_cohort.groupby('subject_id')['admission_age'].first().quantile(0.75):0.1f}")

In [ ]:
g = pd.DataFrame(notes_cohort.groupby('admission_type')['hadm_id'].nunique()).reset_index()
g.columns = ['encounter_type', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)

In [ ]:
g = pd.DataFrame(notes_cohort.groupby('ethnicity')['subject_id'].nunique()).reset_index()
g.columns = ['ethnicity', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)

### Notes Exploration

In [ ]:
notes_df = pd.read_csv(path/'notes_all_proc.csv', parse_dates=['intime', 'admittime', 'ne_charttime'])
model_notes_df = notes_df[notes_df['imi_adm_label'] != -1].reset_index(drop=True)

In [ ]:
print("Encounter time to ICU Admission for model cohort:")
print(f"Mean:{model_notes_df['admit_to_icu'].mean():0.1f}")
print(f"STD:{model_notes_df['admit_to_icu'].std():0.1f}")
print(f"25th percentile:{model_notes_df['admit_to_icu'].quantile(0.25):0.1f}")
print(f"75th percentile:{model_notes_df['admit_to_icu'].quantile(0.75):0.1f}")
print("Encounter time to ICU Admission for notes cohort:")
print(f"Mean:{notes_df['admit_to_icu'].mean():0.1f}")
print(f"STD:{notes_df['admit_to_icu'].std():0.1f}")
print(f"25th percentile:{notes_df['admit_to_icu'].quantile(0.25):0.1f}")
print(f"75th percentile:{notes_df['admit_to_icu'].quantile(0.75):0.1f}")

In [ ]:
print(f"Average Number of clinical notes per encounter for model cohort: {(len(model_notes_df)/model_notes_df['hadm_id'].nunique()):0.1f}")
print(f"Average Number of clinical notes per encounter for notes cohort: {(len(notes_df)/notes_df['hadm_id'].nunique()):0.1f}")

In [ ]:
print("Clinical Note Length for model cohort:")
print(f"Mean:{model_notes_df['note_len'].mean():0.1f}")
print(f"STD:{model_notes_df['note_len'].std():0.1f}")
print(f"25th percentile:{model_notes_df['note_len'].quantile(0.25):0.1f}")
print(f"75th percentile:{model_notes_df['note_len'].quantile(0.75):0.1f}")
print()
print("Clinical Note Length for notes cohort:")
print(f"Mean:{notes_df['note_len'].mean():0.1f}")
print(f"STD:{notes_df['note_len'].std():0.1f}")
print(f"25th percentile:{notes_df['note_len'].quantile(0.25):0.1f}")
print(f"75th percentile:{notes_df['note_len'].quantile(0.75):0.1f}")

In [ ]:
print("Note distribution by category in model cohort:")
g = pd.DataFrame(model_notes_df.groupby('category').size()).reset_index()
g.columns = ['category', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)
print()
print("Note distribution by category in notes cohort:")
g = pd.DataFrame(notes_df.groupby('category').size()).reset_index()
g.columns = ['category', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)

### Notes Plots

In [ ]:
cohort = 'notes_all'
notes_df = pd.read_csv(path/f'{cohort}_proc.csv', parse_dates=['intime', 'admittime', 'ne_charttime'])

save = True

In [ ]:
# Note length distribution
fig, ax = plt.subplots(figsize=(11, 8))
sns.distplot(notes_df['note_len'], kde=False, ax=ax, bins=100)
ax.set_xlim(0, 10000)
ax.set_xlabel('Length of Note (characters)')
ax.set_ylabel('# notes')

if save:
  fig.savefig(figdir/f'{cohort}_note_len_dist.pdf', dpi=300)

In [ ]:
# Note distribution over days before ICU admission binned to 15 days
plot_df = notes_df[['admit_to_icu']]
fig, ax = plt.subplots(figsize=(10, 8))

sns.distplot(plot_df, kde=False, ax=ax, bins=80)
ax.set_xlabel('Time to ICU admission (days)')
ax.set_ylabel('# notes')
ax.set_xlim(0, 70)

if save:
  fig.savefig(figdir/f'{cohort}_admit_to_icu_dist.pdf', dpi=300)

In [ ]:
# Note distribution over days before ICU admission binned to 15 days
intervals = ['-1 ≤ t ≤ 0']
intervals += [f'-{i+1} ≤ t ≤ -{i}' for i in range(1, notes_df['interval'].max())]
intervals.append(f"t ≥ -{notes_df['interval'].max()}")

plot_df = pd.DataFrame(notes_df.loc[notes_df['interval'] != -1].groupby('interval').size(), columns=['n_notes']).reset_index(drop=True)
plot_df['days'] = intervals

fig, ax = plt.subplots(figsize=(15, 8))
sns.barplot(x='days', y='n_notes', data=plot_df, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45, ha='right')
ax.set_xlabel('Time to ICU admission (days)')
ax.set_ylabel('# notes')
for index, row in plot_df.iterrows():
    ax.text(index, row['n_notes'], str(row['n_notes']), color='black', ha='center', va='bottom')

if save:
  fig.savefig(figdir/f'{cohort}_admit_to_icu_binned_dist.pdf', dpi=300)

In [ ]:
# Note distribution over days before ICU admission by Category  binned to 15 days
def plot_intervals(ax, df, cat):
  sns.barplot(x='days', y='n_notes', data=df, ax=ax)
  ax.set_xticklabels(ax.get_xticklabels(),rotation=45, ha='right')
  ax.set_xlabel('')
  ax.set_ylabel('')
  ax.set_title(f"Note Category: {cat}\n# notes: {df['n_notes'].sum()}")   

  for index, (_, row) in enumerate(df.iterrows()):
      ax.text(index, row['n_notes'], str(row['n_notes']), color='black', ha='center', va='bottom') 

plot_df = pd.DataFrame(notes_df.groupby(['category', 'interval']).size(), columns=['n_notes'])
plot_df.reset_index(inplace=True)
plot_df['days'] = plot_df['interval'].apply(lambda x: intervals[x])
plot_df.drop(['interval'], inplace=True, axis=1)

fig, ax = plt.subplots(4, 3, figsize=(20, 25))
plot_intervals(ax[0][0], plot_df.loc[plot_df['category'] == 'Case Management ', ['n_notes', 'days']], 'Case Management')
plot_intervals(ax[0][1], plot_df.loc[plot_df['category'] == 'Consult', ['n_notes', 'days']], 'Consult')
plot_intervals(ax[0][2], plot_df.loc[plot_df['category'] == 'General', ['n_notes', 'days']], 'General')
               
plot_intervals(ax[1][0], plot_df.loc[plot_df['category'] == 'Nursing', ['n_notes', 'days']], 'Nursing')
plot_intervals(ax[1][1], plot_df.loc[plot_df['category'] == 'Nursing/other', ['n_notes', 'days']], 'Nursing/other')
plot_intervals(ax[1][2], plot_df.loc[plot_df['category'] == 'Nutrition', ['n_notes', 'days']], 'Nutrition')

plot_intervals(ax[2][0], plot_df.loc[plot_df['category'] == 'Pharmacy', ['n_notes', 'days']], 'Pharmacy')
plot_intervals(ax[2][1], plot_df.loc[plot_df['category'] == 'Physician ', ['n_notes', 'days',]], 'Physician')
plot_intervals(ax[2][2], plot_df.loc[plot_df['category'] == 'Radiology', ['n_notes', 'days']], 'Radiology')
               
plot_intervals(ax[3][0], plot_df.loc[plot_df['category'] == 'Rehab Services', ['n_notes', 'days']], 'Rehab Services')
plot_intervals(ax[3][1], plot_df.loc[plot_df['category'] == 'Respiratory ', ['n_notes', 'days']], 'Respiratory')
plot_intervals(ax[3][2], plot_df.loc[plot_df['category'] == 'Social Work', ['n_notes', 'days']], 'Social Work')

fig.text(0.5, 0.09, 'Time to ICU admission (days)', ha='center')
fig.text(0.08, 0.5, '# notes', va='center', rotation='vertical')

plt.subplots_adjust(hspace = 0.3)
               
if save:               
  fig.savefig(figdir/f'{cohort}_admit_to_icu_cat_binned_dist.pdf', dpi=300)

In [ ]:
# Histogram of time between note charttime and ICU admittime
plot_df = notes_df[['category', 'note_to_icu']]
fig, ax = plt.subplots(figsize=(10, 8))

sns.distplot(plot_df['note_to_icu'], kde=False, ax=ax, bins=80)
ax.set_xlabel('Note Charttime to ICU Admittime (days)')
ax.set_ylabel('# notes')
ax.set_xlim(0, 60)

if save:
  fig.savefig(figdir/f'{cohort}_note_to_icu_dist.pdf', dpi=300)

In [ ]:
# Histogram of time between note charttime and ICU admittime by Category
def plot_period(ax, df, cat):
  sns.distplot(df, kde=False, ax=ax, bins=10)
  ax.set_xlabel('')
  ax.set_ylabel('')
  ax.set_title(f"Note Category: {cat}")

fig, ax = plt.subplots(4, 3, figsize=(20, 25))
plot_period(ax[0][0], plot_df.loc[plot_df['category'] == 'Case Management ', ['note_to_icu']], 'Case Management')
plot_period(ax[0][1], plot_df.loc[plot_df['category'] == 'Consult', ['note_to_icu']], 'Consult')
plot_period(ax[0][2], plot_df.loc[plot_df['category'] == 'General', ['note_to_icu']], 'General')

plot_period(ax[1][0], plot_df.loc[plot_df['category'] == 'Nursing', ['note_to_icu']], 'Nursing')
plot_period(ax[1][1], plot_df.loc[plot_df['category'] == 'Nursing/other', ['note_to_icu']], 'Nursing/other')
plot_period(ax[1][2], plot_df.loc[plot_df['category'] == 'Nutrition', ['note_to_icu']], 'Nutrition')

plot_period(ax[2][0], plot_df.loc[plot_df['category'] == 'Pharmacy', ['note_to_icu']], 'Pharmacy')
plot_period(ax[2][1], plot_df.loc[plot_df['category'] == 'Physician ', ['note_to_icu',]], 'Physician')
plot_period(ax[2][2], plot_df.loc[plot_df['category'] == 'Radiology', ['note_to_icu']], 'Radiology')

plot_period(ax[3][0], plot_df.loc[plot_df['category'] == 'Rehab Services', ['note_to_icu']], 'Rehab Services')
plot_period(ax[3][1], plot_df.loc[plot_df['category'] == 'Respiratory ', ['note_to_icu']], 'Respiratory')
plot_period(ax[3][2], plot_df.loc[plot_df['category'] == 'Social Work', ['note_to_icu']], 'Social Work')

fig.text(0.5, 0.1, 'Note Charttime to ICU Admittime (days)', ha='center')
fig.text(0.08, 0.5, '# notes', va='center', rotation='vertical')

plt.subplots_adjust(hspace = 0.1)

if save:
  fig.savefig(figdir/f'{cohort}_note_to_icu_cat_dist.pdf', dpi=300)

In [ ]:
desc = ['Unused', 'Delayed ICU Admission', 'Imminent ICU Admission']

p = pd.DataFrame(notes_df.groupby(['imi_adm_label']).size(), columns=['n_notes']).reset_index()
# p1 = pd.DataFrame(notes_df.groupby(['imi_adm_label']).size(), columns=['n_notes']).reset_index()
# p2 = notes_df.groupby(['imi_adm_label'])['hadm_id'].nunique().reset_index()

# p = p1.merge(p2, on=['imi_adm_label'])
p['imi_adm_label'] = desc
p = p.reindex([2, 1, 0])
# p.reset_index(inplace=True, drop=True)

plot_df = p.copy()
plot_df.rename(columns={'hadm_id':'# Encounters', 'n_notes':'# Notes'}, inplace=True)
plot_df = pd.melt(plot_df, id_vars='imi_adm_label', var_name='Legend', value_name='counts')

plot_df

fig, ax = plt.subplots(figsize=(11, 8))
sns.barplot(x='imi_adm_label', y='counts', data=plot_df, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='center')
ax.set_xlabel('Class Label')
ax.set_ylabel('# notes')

for index, row in plot_df.iterrows():
#     if index < len(plot_df)//2:
        ax.text(index+0.06, row['counts'], str(row['counts']), color='black', ha='right', va='bottom')
#     else:
#         ax.text(index % (len(plot_df)//2), row['counts'], str(row['counts']), color='black', ha='right', va='bottom')

if save:
  fig.savefig(figdir/f'{cohort}_note_class_dist.pdf', dpi=300)

## Data Processing

In [ ]:
# 1. Read in the data and drop duplicates
notes_df = pd.read_csv(mimic_path/'imi_notes_mimic.csv', parse_dates=['intime', 'admittime', 'ne_charttime'])
notes_df.drop_duplicates(inplace=True)

str_df = pd.read_csv(mimic_path/'imi_str_mimic.csv', parse_dates=['ce_charttime'])
str_df.drop_duplicates(inplace=True)

str_df.shape, notes_df.shape

Get the `hadm_id`s for just the notes and create table for those. This is to get the characterstics only those `hadm_ids` with notes (and possibly structured) for extracting cohort characterstics in case just notes results suck.

```
create table notes_adms (hadm_id integer);
\copy notes_adms(hadm_id) from '/path/to/notes_hadm_ids.csv' delimiter ',' csv header;
```

In [ ]:
with open(mimic_path/'hadm_ids_with_notes.csv', 'w') as f:
  f.write('hadm_id\n')
  f.write('\n'.join([str(i) for i in notes_df['hadm_id'].unique()]))

In [ ]:
# 2. Extract common `hadm_id` and filter out those that do not appear in both dfs
hadms = set(str_df['hadm_id'].unique()).intersection(notes_df['hadm_id'].unique())
print(f"Number of encounters having both forms of data: {len(hadms)}")

common_notes = notes_df[notes_df['hadm_id'].isin(hadms)].reset_index(drop=True)
common_str = str_df[str_df['hadm_id'].isin(hadms)].reset_index(drop=True)

common_str.shape, str_df.shape, common_notes.shape, notes_df.shape

Create a new table with just the common `hadm_id` by running the following code:
```
create table common_adms (hadm_id integer);
\copy common_adms(hadm_id) from '/path/to/common_hadm_ids.csv' delimiter ',' csv header;
```

In [ ]:
# sanity check and write the common `hadm_id`s to disk for extracting cohort characterstics
s, n = set(common_str['hadm_id'].unique()), set(common_notes['hadm_id'].unique())
assert(s.symmetric_difference(n) == set())

with open(mimic_path/'common_hadm_ids.csv', 'w') as f:
  f.write('hadm_id\n')
  f.write('\n'.join([str(i) for i in s]))

After this we run `cohort.sql` from the MIMIC extraction repo, to create the cohort table which contains details about the cohort. We will have two tables: `common_cohort` with `hadm_id`s that have both structured and notes data, and `notes_cohort` which with `hadm_id`s that have notes data (and maybe sturctured data that we don't care about).

In [ ]:
# 3. remove redundant info by filling in each time column with the value of the var
common_str = common_str.groupby(['hadm_id','ce_charttime']).sum(min_count = 1).reset_index()
# 4. groupby ffill 
common_str = common_str.groupby(['hadm_id'], as_index=False).apply(lambda group: group.ffill())
common_str.shape

In [ ]:
# Steps 5 & 6 for both common_notes and notes_df
# 5. cat the category, description, text into a new note col and remove them
notes_df['note'] = notes_df['category'].str.cat(notes_df['description'], sep='\n')
notes_df['note'] = notes_df['note'].str.cat(notes_df['text'], sep='\n')
notes_df['note_len'] = notes_df['note'].apply(len)
notes_df.drop(columns=['category', 'description', 'text'], inplace=True)  

# 6. cat notes charted at the same time
notes_df = pd.DataFrame(notes_df.groupby(['hadm_id', 'intime', 'admittime', 'ne_charttime'])['note'].apply('\n'.join)).reset_index()

notes_df['category'] = notes_df['note'].apply(lambda x: x.split('\n')[0])
notes_df['admit_to_icu'] = (notes_df['intime'] - notes_df['admittime'])/np.timedelta64(1, 'D')
notes_df['note_to_icu'] = (notes_df['intime'] - notes_df['ne_charttime'])/np.timedelta64(1, 'D')
notes_df['note_len'] = notes_df['note'].apply(len)

# 5. cat the category, description, text into a new note col and remove them
common_notes['note'] = common_notes['category'].str.cat(common_notes['description'], sep='\n')
common_notes['note'] = common_notes['note'].str.cat(common_notes['text'], sep='\n')
common_notes['note_len'] = common_notes['note'].apply(len)
common_notes.drop(columns=['category', 'description', 'text'], inplace=True)  

# 6. cat notes charted at the same time
common_notes = pd.DataFrame(common_notes.groupby(['hadm_id', 'intime', 'admittime', 'ne_charttime'])['note'].apply('\n'.join)).reset_index()

common_notes['category'] = common_notes['note'].apply(lambda x: x.split('\n')[0])
common_notes['admit_to_icu'] = (common_notes['intime'] - common_notes['admittime'])/np.timedelta64(1, 'D')
common_notes['note_to_icu'] = (common_notes['intime'] - common_notes['ne_charttime'])/np.timedelta64(1, 'D')
common_notes['note_len'] = common_notes['note'].apply(len)

common_notes.shape, notes_df.shape, notes_df.columns

We can drop those rows which don't have basic vitals of HR, MAP, and RESP. But for now, we go with everything we have.

In [ ]:
# common_str.dropna(subset=['hr', 'map', 'resp'], how='any', inplace=True)

In [ ]:
common_str.to_csv(mimic_path/'proc1_str_mimic.csv', index=False)
common_notes.to_csv(mimic_path/'proc1_notes_mimic_common.csv', index=False)
notes_df.to_csv(mimic_path/'proc1_notes_mimic.csv', index=False)

### Dev for Structured Data Stats

This code is only development on a subset of encounters to calculate change statistics. This will be run for all the encounters separately using the `stats.py` script

In [ ]:
def percentile(n):
  def percentile_(x):
    return x.quantile(n)
  percentile_.__name__ = f'percentile_{n*100:2.0f}'
  return percentile_

def change_name(col_name):
  if '(' not in col_name:
    return col_name
  cols = literal_eval(col_name)
  return f'{cols[0]}_{cols[1]}'

statistics = ['min', 'mean', 'median', 'std', 'var', 'kurt', 'skew', percentile(0.25), percentile(0.75), stats.iqr, 'max']

In [ ]:
common_str = pd.read_csv(mimic_path/'proc1_str_mimic.csv', parse_dates=['ce_charttime'])
var_cols = common_str.columns[2:]

In [ ]:
hadms = [100104, 100975, 101511, 111073]
subset_df = common_str.loc[(common_str['hadm_id'].isin(hadms))].reset_index(drop=True)
subset_df.groupby('hadm_id').size()

In [ ]:
for hadm_id, group_df in tqdm(subset_df.groupby('hadm_id'), desc='Encounters'):
  df = group_df.copy()
  var_df = df[var_cols].reset_index(drop=True) # save the original vals for later
  
  df.set_index('ce_charttime', inplace=True) # set charttime as index for rolling 24h
  stats_df = df[var_cols].rolling('24h').agg(statistics)
  
  df = pd.DataFrame(stats_df.to_records()) # flatten the resulting dataframe
  df.insert(loc=1, column='hadm_id', value=hadm_id)
  
  df.rename(columns=change_name, inplace=True) # rename columns
  df = pd.concat([df, var_df], axis=1) # add the original vals back
  
  # reorder vars such that the columns are var, var_stat...
  stats_cols = df.columns[2:]
  all_cols = []
  for var in var_cols:
    all_cols.append(var)
    for stat in stats_cols:
      if f'{var}_' in stat:
        all_cols.append(stat)
        
  order = list(df.columns[:2]) + all_cols
  df = df[order]
  
  df.to_csv(mimic_path/f'stats_dir/{hadm_id}.csv', index=False)    

In [ ]:
dfs = []
stats_dir = mimic_path/'stats_dir'
n_files = 0
for _ in stats_dir.glob('*.csv'):
  n_files += 1

for i, f in enumerate(tqdm(stats_dir.glob('*.csv'), total=n_files, desc='Stats CSV')):
    dfs.append(pd.read_csv(f))

common_str = pd.concat(dfs)
common_str.reset_index(drop=True, inplace=True)
common_str['ce_charttime'] =  pd.to_datetime(common_str['ce_charttime'])
common_str.to_csv(mimic_path/'proc2_str_mimic_stats.csv', index=False)

## Data Labeling

## Data Exploration

We are going to label only the notes df. Since we are basing our time element on `ne_charttime` and attaching structured data to `common_notes` to get final merged df, we don't need to label structured data separately.

In [ ]:
def note_interval(x):
  if pd.isnull(x):
    return -1
  
  if 0 < x <= 1:
    return 0
  elif 1 < x <= 2:
    return 1
  elif 2 < x <= 3:
    return 2
  elif 3 < x <= 4:
    return 3
  elif 4 < x <= 5:
    return 4
  elif 5 < x <= 6:
    return 5
  elif 6 < x <= 7:
    return 6
  elif 7 < x <= 8:
    return 7
  elif 8 < x <= 9:
    return 8
  elif 9 < x <= 10:
    return 9
  elif 10 < x <= 11:
    return 10
  elif 11 < x <= 12:
    return 11
  elif 12 < x <= 13:
    return 12
  elif 13 < x <= 14:
    return 13
  elif 14 < x <= 15:
    return 14
  else:
    return 15

def icu_adm_label(x):
  if 0 <= x <= 1:
    return -1 # unused notes due to data leakage
  elif 1 < x <= 3:
    return 1 # imminent ICU admission
  elif 3 < x <= 5:
    return -1 # unused notes due to data leakage
  else:
    return 0 # delayed ICU admission

In [ ]:
notes_df = pd.read_csv(mimic_path/'proc1_notes_mimic.csv', parse_dates=['intime', 'admittime', 'ne_charttime'])
common_notes = pd.read_csv(mimic_path/'proc1_notes_mimic_common.csv', parse_dates=['intime', 'admittime', 'ne_charttime'])
common_notes.shape, notes_df.shape, notes_df.columns

In [ ]:
notes_df['interval'] = notes_df['note_to_icu'].apply(note_interval)
common_notes['interval'] = common_notes['note_to_icu'].apply(note_interval)
notes_df['imi_adm_label'] = notes_df['interval'].apply(icu_adm_label)
common_notes['imi_adm_label'] = common_notes['interval'].apply(icu_adm_label)

In [ ]:
g = common_notes.loc[common_notes['imi_adm_label'] != -1].groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence in notes cohort:{(g[1]/g.sum())*100:0.1f}")
g = notes_df.loc[notes_df['imi_adm_label'] != -1].groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence in notes cohort:{(g[1]/g.sum())*100:0.1f}")

In [ ]:
notes_df.to_csv(mimic_path/'proc3_notes_mimic_labeled.csv', index=False)
common_notes.to_csv(mimic_path/'proc3_notes_mimic_common_labeled.csv', index=False)

### Cohort Characterstics

In [ ]:
common_cohort = pd.read_csv(mimic_path/'common_cohort.csv')
notes_cohort = pd.read_csv(mimic_path/'notes_cohort.csv')
common_cohort.shape, notes_cohort.shape

In [ ]:
def group_eth(eth):
  eth = eth.lower()
  if 'white' in eth:
    return 'white'
  elif 'black' in eth:
    return 'black'
  elif 'hispanic' in eth:
    return 'hispanic'
  elif 'asian' in eth:
    return 'asian'
  else:
    return 'other'

common_cohort['ethnicity'] = common_cohort['ethnicity'].apply(group_eth)
notes_cohort['ethnicity'] = notes_cohort['ethnicity'].apply(group_eth)
common_cohort.loc[common_cohort['admission_age'] > 100, 'admission_age'] = 100
notes_cohort.loc[notes_cohort['admission_age'] > 100, 'admission_age'] = 100

In [ ]:
print(f"Number of patients in common cohort: {common_cohort['subject_id'].nunique()}")
print(f"Number of patients in notes cohort: {notes_cohort['subject_id'].nunique()}")

In [ ]:
g = common_cohort.groupby('expire_flag')['subject_id'].nunique().to_numpy()
print(f"Mortality in common cohort: {g[1]} ({(g[1]/g.sum())*100:0.1f}%)")

g = notes_cohort.groupby('expire_flag')['subject_id'].nunique().to_numpy()
print(f"Mortality in notes cohort: {g[1]} ({(g[1]/g.sum())*100:0.1f}%)")

In [ ]:
g = common_cohort.groupby('gender')['subject_id'].nunique().to_numpy()
print(f"Males in common cohort: {g[1]} ({(g[1]/g.sum())*100:0.1f}%)")
g = notes_cohort.groupby('gender')['subject_id'].nunique().to_numpy()
print(f"Males in notes cohort: {g[1]} ({(g[1]/g.sum())*100:0.1f}%)")

In [ ]:
print("Age details in common cohort:")
print(f"Mean:{common_cohort.groupby('subject_id')['admission_age'].first().mean():0.1f}")
print(f"STD:{common_cohort.groupby('subject_id')['admission_age'].first().std():0.1f}")
print(f"25th percentile:{common_cohort.groupby('subject_id')['admission_age'].first().quantile(0.25):0.1f}")
print(f"75th percentile:{common_cohort.groupby('subject_id')['admission_age'].first().quantile(0.75):0.1f}")
print()
print("Age details in notes cohort:")
print(f"Mean:{notes_cohort.groupby('subject_id')['admission_age'].first().mean():0.1f}")
print(f"STD:{notes_cohort.groupby('subject_id')['admission_age'].first().std():0.1f}")
print(f"25th percentile:{notes_cohort.groupby('subject_id')['admission_age'].first().quantile(0.25):0.1f}")
print(f"75th percentile:{notes_cohort.groupby('subject_id')['admission_age'].first().quantile(0.75):0.1f}")

In [ ]:
print("Admission types for common cohort:")
g = pd.DataFrame(common_cohort.groupby('admission_type')['hadm_id'].nunique()).reset_index()
g.columns = ['encounter_type', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)
print()
print("Admission types for notes cohort:")
g = pd.DataFrame(notes_cohort.groupby('admission_type')['hadm_id'].nunique()).reset_index()
g.columns = ['encounter_type', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)

In [ ]:
print("Ethnicty for common cohort:")
g = pd.DataFrame(common_cohort.groupby('ethnicity')['subject_id'].nunique()).reset_index()
g.columns = ['ethnicity', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)
print()
print("Ethnicty for notes cohort:")
g = pd.DataFrame(notes_cohort.groupby('ethnicity')['subject_id'].nunique()).reset_index()
g.columns = ['ethnicity', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)

### Notes Exploration

In [ ]:
notes_df = pd.read_csv(mimic_path/'proc3_notes_mimic_labeled.csv', parse_dates=['intime', 'admittime', 'ne_charttime'])
common_notes = pd.read_csv(mimic_path/'proc3_notes_mimic_common_labeled.csv', parse_dates=['intime', 'admittime', 'ne_charttime'])
common_notes.shape, notes_df.shape, notes_df.columns

In [ ]:
print("Encounter time to ICU Admission for common cohort:")
print(f"Mean:{common_notes['admit_to_icu'].mean():0.1f}")
print(f"STD:{common_notes['admit_to_icu'].std():0.1f}")
print(f"25th percentile:{common_notes['admit_to_icu'].quantile(0.25):0.1f}")
print(f"75th percentile:{common_notes['admit_to_icu'].quantile(0.75):0.1f}")
print()
print("Encounter time to ICU Admission for notes cohort:")
print(f"Mean:{notes_df['admit_to_icu'].mean():0.1f}")
print(f"STD:{notes_df['admit_to_icu'].std():0.1f}")
print(f"25th percentile:{notes_df['admit_to_icu'].quantile(0.25):0.1f}")
print(f"75th percentile:{notes_df['admit_to_icu'].quantile(0.75):0.1f}")

In [ ]:
print(f"Average Number of clinical notes per encounter for common cohort: {(len(common_notes)/common_notes['hadm_id'].nunique()):0.1f}")
print(f"Average Number of clinical notes per encounter for notes cohort: {(len(notes_df)/notes_df['hadm_id'].nunique()):0.1f}")

In [ ]:
print("Clinical Note Length for common cohort:")
print(f"Mean:{common_notes['note_len'].mean():0.1f}")
print(f"STD:{common_notes['note_len'].std():0.1f}")
print(f"25th percentile:{common_notes['note_len'].quantile(0.25):0.1f}")
print(f"75th percentile:{common_notes['note_len'].quantile(0.75):0.1f}")
print()
print("Clinical Note Length for notes cohort:")
print(f"Mean:{notes_df['note_len'].mean():0.1f}")
print(f"STD:{notes_df['note_len'].std():0.1f}")
print(f"25th percentile:{notes_df['note_len'].quantile(0.25):0.1f}")
print(f"75th percentile:{notes_df['note_len'].quantile(0.75):0.1f}")

In [ ]:
print("Note distribution by category in common cohort:")
g = pd.DataFrame(common_notes.groupby('category').size()).reset_index()
g.columns = ['category', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)
print()
print("Note distribution by category in notes cohort:")
g = pd.DataFrame(notes_df.groupby('category').size()).reset_index()
g.columns = ['category', 'count']
g['pct'] = np.round((g['count']/g['count'].sum() * 100), 1)
print(g)

### Notes Plots

In [ ]:
save = True
cohort = 'common'
figdir = mimic_path/'figures'

if cohort == 'common':
  df = common_notes
elif cohort == 'notes':
  df = notes_df

In [ ]:
# Note length distribution
fig, ax = plt.subplots(figsize=(10, 8))
sns.distplot(df['note_len'], kde=False, ax=ax, bins=100)
ax.set_xlim(0, 12500)
ax.set_xlabel('Length of Note (characters)')
ax.set_ylabel('# notes')

if save:
  fig.savefig(figdir/f'{cohort}_note_len_dist.pdf', dpi=300)

In [ ]:
# Note distribution over days before ICU admission binned to 15 days
plot_df = df[['admit_to_icu']]
fig, ax = plt.subplots(figsize=(10, 8))

sns.distplot(plot_df, kde=False, ax=ax, bins=80)
ax.set_xlabel('Time to ICU admission (days)')
ax.set_ylabel('# notes')
ax.set_xlim(0, 70)

if save:
  fig.savefig(figdir/f'{cohort}_to_icu_dist.pdf', dpi=300)

In [ ]:
# Note distribution over days before ICU admission binned to 15 days
intervals = ['-1 ≤ t ≤ 0']
intervals += [f'-{i+1} ≤ t ≤ -{i}' for i in range(1, notes_df['interval'].max())]
intervals.append(f"t ≥ -{df['interval'].max()}")

plot_df = pd.DataFrame(df.loc[notes_df['interval'] != -1].groupby('interval').size(), columns=['n_notes']).reset_index(drop=True)
plot_df['days'] = intervals

fig, ax = plt.subplots(figsize=(15, 8))
sns.barplot(x='days', y='n_notes', data=plot_df, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45, ha='right')
ax.set_xlabel('Time to ICU admission (days)')
ax.set_ylabel('# notes')
for index, row in plot_df.iterrows():
    ax.text(index, row['n_notes'], str(row['n_notes']), color='black', ha='center', va='bottom')

if save:
  fig.savefig(figdir/f'{cohort}_admit_to_icu_binned_dist.pdf', dpi=300)

In [ ]:
# Note distribution over days before ICU admission by Category  binned to 15 days
def plot_intervals(ax, df, cat):
  sns.barplot(x='days', y='n_notes', data=df, ax=ax)
  ax.set_xticklabels(ax.get_xticklabels(),rotation=45, ha='right')
  ax.set_xlabel('')
  ax.set_ylabel('')
  ax.set_title(f"Note Category: {cat}\n# notes: {df['n_notes'].sum()}")   

  for index, (_, row) in enumerate(df.iterrows()):
      ax.text(index, row['n_notes'], str(row['n_notes']), color='black', ha='center', va='bottom') 

plot_df = pd.DataFrame(df.groupby(['category', 'interval']).size(), columns=['n_notes'])
plot_df.reset_index(inplace=True)
plot_df['days'] = plot_df['interval'].apply(lambda x: intervals[x])
plot_df.drop(['interval'], inplace=True, axis=1)

fig, ax = plt.subplots(6, 2, figsize=(20, 50))
plot_intervals(ax[0][0], plot_df.loc[plot_df['category'] == 'Case Management ', ['n_notes', 'days']], 'Case Management')
plot_intervals(ax[0][1], plot_df.loc[plot_df['category'] == 'Consult', ['n_notes', 'days']], 'Consult')

plot_intervals(ax[1][0], plot_df.loc[plot_df['category'] == 'General', ['n_notes', 'days']], 'General')
plot_intervals(ax[1][1], plot_df.loc[plot_df['category'] == 'Nursing', ['n_notes', 'days']], 'Nursing')

plot_intervals(ax[2][0], plot_df.loc[plot_df['category'] == 'Nursing/other', ['n_notes', 'days']], 'Nursing/other')
plot_intervals(ax[2][1], plot_df.loc[plot_df['category'] == 'Nutrition', ['n_notes', 'days']], 'Nutrition')

plot_intervals(ax[3][0], plot_df.loc[plot_df['category'] == 'Pharmacy', ['n_notes', 'days']], 'Pharmacy')
plot_intervals(ax[3][1], plot_df.loc[plot_df['category'] == 'Physician ', ['n_notes', 'days',]], 'Physician')

plot_intervals(ax[4][0], plot_df.loc[plot_df['category'] == 'Radiology', ['n_notes', 'days']], 'Radiology')
plot_intervals(ax[4][1], plot_df.loc[plot_df['category'] == 'Rehab Services', ['n_notes', 'days']], 'Rehab Services')

plot_intervals(ax[5][0], plot_df.loc[plot_df['category'] == 'Respiratory ', ['n_notes', 'days']], 'Respiratory')
plot_intervals(ax[5][1], plot_df.loc[plot_df['category'] == 'Social Work', ['n_notes', 'days']], 'Social Work')

fig.text(0.5, 0.1, 'Time to ICU admission (days)', ha='center')
fig.text(0.08, 0.5, '# notes', va='center', rotation='vertical')

plt.subplots_adjust(hspace = 0.3)
               
if save:               
  fig.savefig(figdir/f'{cohort}_admit_to_icu_cat_binned_dist.pdf', dpi=300)

In [ ]:
# Histogram of time between note charttime and ICU admittime
plot_df = df[['category', 'note_to_icu']]
fig, ax = plt.subplots(figsize=(10, 8))

sns.distplot(plot_df['note_to_icu'], kde=False, ax=ax, bins=80)
ax.set_xlabel('Note Charttime to ICU Admittime (days)')
ax.set_ylabel('# notes')
ax.set_xlim(0, 60)

if save:
  fig.savefig(figdir/f'{cohort}_note_to_icu_dist.pdf', dpi=300)

In [ ]:
# Histogram of time between note charttime and ICU admittime by Category
def plot_period(ax, df, cat):
  sns.distplot(df, kde=False, ax=ax, bins=10)
  ax.set_xlabel('')
  ax.set_ylabel('')
  ax.set_title(f"Note Category: {cat}")

fig, ax = plt.subplots(6, 2, figsize=(20, 50))
plot_period(ax[0][0], plot_df.loc[plot_df['category'] == 'Case Management ', ['note_to_icu']], 'Case Management')
plot_period(ax[0][1], plot_df.loc[plot_df['category'] == 'Consult', ['note_to_icu']], 'Consult')

plot_period(ax[1][0], plot_df.loc[plot_df['category'] == 'General', ['note_to_icu']], 'General')
plot_period(ax[1][1], plot_df.loc[plot_df['category'] == 'Nursing', ['note_to_icu']], 'Nursing')

plot_period(ax[2][0], plot_df.loc[plot_df['category'] == 'Nursing/other', ['note_to_icu']], 'Nursing/other')
plot_period(ax[2][1], plot_df.loc[plot_df['category'] == 'Nutrition', ['note_to_icu']], 'Nutrition')

plot_period(ax[3][0], plot_df.loc[plot_df['category'] == 'Pharmacy', ['note_to_icu']], 'Pharmacy')
plot_period(ax[3][1], plot_df.loc[plot_df['category'] == 'Physician ', ['note_to_icu',]], 'Physician')

plot_period(ax[4][0], plot_df.loc[plot_df['category'] == 'Radiology', ['note_to_icu']], 'Radiology')
plot_period(ax[4][1], plot_df.loc[plot_df['category'] == 'Rehab Services', ['note_to_icu']], 'Rehab Services')

plot_period(ax[5][0], plot_df.loc[plot_df['category'] == 'Respiratory ', ['note_to_icu']], 'Respiratory')
plot_period(ax[5][1], plot_df.loc[plot_df['category'] == 'Social Work', ['note_to_icu']], 'Social Work')

fig.text(0.5, 0.11, 'Note Charttime to ICU Admittime (days)', ha='center')
fig.text(0.08, 0.5, '# notes', va='center', rotation='vertical')

plt.subplots_adjust(hspace = 0.1)

if save:
  fig.savefig(figdir/f'{cohort}_note_to_icu_cat_dist.pdf', dpi=300)

In [ ]:
desc = ['Delayed ICU Admission', 'Imminent ICU Admission']

p1 = pd.DataFrame(df.loc[df['imi_adm_label'] != -1].groupby(['imi_adm_label']).size(), columns=['n_notes']).reset_index()
p2 = df.loc[df['imi_adm_label'] != -1].groupby(['imi_adm_label'])['hadm_id'].nunique().reset_index()

p = p1.merge(p2, on=['imi_adm_label'])
p['imi_adm_label'] = desc
p = p.reindex([1, 0])
p.reset_index(inplace=True, drop=True)

plot_df = p.copy()
plot_df.rename(columns={'hadm_id':'# Encounters', 'n_notes':'# Notes'}, inplace=True)
plot_df = pd.melt(plot_df, id_vars='imi_adm_label', var_name='Legend', value_name='counts')

fig, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x='imi_adm_label', y='counts', hue='Legend', data=plot_df, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='center')
ax.set_xlabel('Class Label')
ax.set_ylabel('# notes')

for index, row in plot_df.iterrows():
    if index < len(plot_df)//2:
        ax.text(index-0.13, row['counts']+50, str(row['counts']), color='black', ha='right', va='bottom')
    else:
        ax.text(index % (len(plot_df)//2)+0.25, row['counts']+50, str(row['counts']), color='black', ha='right', va='bottom')

if save:
  fig.savefig(figdir/f'{cohort}_note_class_dist.pdf', dpi=300)

## Merge Structured and Notes

In [ ]:
common_notes = pd.read_csv(mimic_path/'proc3_notes_mimic_common_labeled.csv', parse_dates=['intime', 'admittime', 'ne_charttime'])
common_str = pd.read_csv(mimic_path/'proc2_str_mimic_stats.csv', parse_dates=['ce_charttime'])

s, n = set(common_str['hadm_id'].unique()), set(common_notes['hadm_id'].unique())
assert(s.symmetric_difference(n) == set())
col_order = ['hadm_id', 'intime', 'admittime', 'admit_to_icu'] + list(common_str.columns[2:]) + ['ne_charttime', 'note_to_icu', 'interval', 'category', 'note', 'note_len', 'imi_adm_label']

common_str.shape, common_notes.shape, len(col_order)

In [ ]:
common_str.sort_values(by='ce_charttime', inplace=True)
common_str.reset_index(inplace=True, drop=True)

common_notes.sort_values(by='ne_charttime', inplace=True)
common_notes.reset_index(inplace=True, drop=True)

In [ ]:
mimic_df = pd.merge_asof(common_notes, common_str, left_on='ne_charttime', right_on='ce_charttime', by='hadm_id')
mimic_df.drop(columns='ce_charttime', inplace=True)
mimic_df = mimic_df[col_order]

mimic_df.shape

In [ ]:
mimic_df.to_csv(mimic_path/'merged_labeled_mimic.csv', index=False)

## Stats

In [ ]:
all_str_df = pd.read_csv(mimic_path/'proc1_str_mimic.csv', parse_dates=['ce_charttime'])
all_var_cols = all_str_df.columns[2:]

In [ ]:
hadms = [100104, 100975, 101511, 111073]
dev_str = all_str_df.loc[(all_str_df['hadm_id'].isin(hadms))].reset_index(drop=True)
dev_str.shape

In [ ]:
def percentile(n):
  def percentile_(x):
    return x.quantile(n)
  percentile_.__name__ = f'percentile_{n*100:2.0f}'
  return percentile_

def all_change_name(col_name):
  if '(' not in col_name:
    return col_name
  cols = literal_eval(col_name)
  return f'{cols[2]}_{cols[1]}'

In [ ]:
statistics = ['min', 'mean', 'median', 'std', 'var', 'kurt', 'skew', percentile(0.25), percentile(0.75), stats.iqr, 'max']

In [ ]:
all_var_df = dev_str[all_var_cols] # save the original vals for later
dev_str.set_index('ce_charttime', inplace=True) # set charttime index for 24h rolling
dev_str.shape

In [ ]:
all_stats_df = dev_str.groupby('hadm_id')[all_var_cols].rolling('24h').agg(statistics)
all_stats_df.shape

In [ ]:
dev_str = pd.DataFrame(all_stats_df.to_records()) # flatten the resulting dataframe
dev_str.shape

In [ ]:
dev_str = dev_str.iloc[:, :(2 + (len(statistics) * len(all_var_cols)))] # drop duplicate columns resulting from rolling
dev_str.shape

In [ ]:
dev_str.rename(columns=all_change_name, inplace=True) # rename columns
dev_str.shape

In [ ]:
dev_str = pd.concat([dev_str, all_var_df], axis=1) # add the original vals back
dev_str.shape

In [ ]:
# reorder vars such that the columns are var, var_stat...
stats_cols = dev_str.columns[2:]
all_cols = []
for var in all_var_cols:
  all_cols.append(var)
  for stat in stats_cols:
    if f'{var}_' in stat:
      all_cols.append(stat)

In [ ]:
order = list(dev_str.columns[:2]) + all_cols
dev_str = dev_str[order]
dev_str.shape

In [ ]:
dev_str

In [ ]:
dev_str.to_csv(mimic_path/'sample_str.csv', index=False)

## Checkpoint

In [ ]:
cols = list(str_df.columns[1:]) # get the cols to merge (everything except hadm_id)
final_dfs = [] 

grouped = notes_df.groupby('hadm_id') # get groups of encounter ids
for name, group in grouped:
  final_df = group.copy().reset_index(drop=True) # make a copy of notes for that encounter
  for col in cols:
    final_df[col] = np.nan # set the values to nan

  idx = 0 # index to track the final row in the given encounter
  for i, note_row in final_df.iterrows():
    ne = note_row['ne_charttime']
    sub = str_df.loc[(str_df['hadm_id'] == name)].reset_index(drop=True) # get the df corresponding to the ecounter
    for j, str_row in sub.iterrows():
      ce = str_row['ce_charttime']
      if ne < ce: # if the variable charttime < note charttime
        idx += 1
        
        # grab the previous values for the variables and break
        for col in cols:
          final_df.iloc[i, final_df.columns.get_loc(col)] = sub.iloc[j-1][col]          
        break               
  pdb.set_trace()
  # get the last value in the df for the variables
  for col in cols:
    final_df.iloc[idx, final_df.columns.get_loc(col)] = sub.iloc[-1][col]
  
  final_dfs.append(final_df) # append the df to the list

In [ ]:
# cat the list to get final df and reset index
mimic_df = pd.concat(final_dfs)
mimic_df.reset_index(inplace=True, drop=True)

## Checkpoint

In [ ]:
dev = True
if dev:
  columns = ['hadm_id', 'ce_charttime', 'hr', 'resp', 'magnesium']
  var_cols = columns[2:]
  working_hadms = [196673, 197006]
  str_df = pd.read_csv(mimic_path/'proc1_str_mimic.csv', usecols=columns, parse_dates=['ce_charttime'])
else:
  working_hadms = str_df['hadm_id'].unique()  
  str_df = pd.read_csv(mimic_path/'proc1_str_mimic.csv', parse_dates=['ce_charttime'])
  var_cols = str_df.columns[2:]

In [ ]:
dev_str = str_df.loc[(str_df['hadm_id'].isin(working_hadms))].reset_index(drop=True)
dev_notes = notes_df.loc[(notes_df['hadm_id'].isin(working_hadms))][['hadm_id', 'ne_charttime', 'note']].reset_index(drop=True)
dev_notes.sort_values('ne_charttime', inplace=True)

In [ ]:
# remove redundant info by filling in each time column with the value of the var
dev_str = dev_str.groupby(['hadm_id','ce_charttime']).sum(min_count = 1).reset_index()

# groupby ffill 
dev_str = dev_str.groupby(['hadm_id'], as_index=False).apply(lambda group: group.ffill())
var_cols = dev_str.columns[2:]

In [ ]:
def percentile(n):
  def percentile_(x):
    return x.quantile(n)
  percentile_.__name__ = f'percentile_{n*100:2.0f}'
  return percentile_

def get_stats(df, var_cols, statistics):
  df.set_index('ce_charttime', inplace=True)
  stats_dfs = []

  for var in var_cols:
    stats_df = df.groupby('hadm_id')[var].rolling('24h').agg(statistics).reset_index(drop=True)
    stats_df.columns = [f'{var}_{col}' for col in stats_df.columns]
    stats_df = pd.concat([df[var].reset_index(drop=True), stats_df], axis=1)
    stats_dfs.append(stats_df)
    
  df.reset_index(inplace=True)
  df.drop(var_cols, inplace=True, axis=1)
  return pd.concat([df, *stats_dfs], axis=1)

In [ ]:
statistics = ['min', 'mean', 'median', 'std', 'var', 'kurt', 'skew', percentile(0.25), percentile(0.75), stats.iqr, 'max']

dev_str = get_stats(dev_str.copy(), var_cols, statistics)
move = ['hadm_id', 'ce_charttime']
order = move + (dev_str.columns.drop(move).tolist())
dev_str = dev_str[order]

In [ ]:
df_str = dev_str.loc[(dev_str['hadm_id'] == 196673)].reset_index(drop=True)
df_notes = dev_notes.loc[(dev_notes['hadm_id'] == 196673)].reset_index(drop=True)
df_str.shape, df_notes.shape

In [ ]:
df_str

In [ ]:
df_notes

In [ ]:
cols = list(dev_str.columns[1:]) # get the cols to merge (everything except hadm_id)
final_dfs = [] 

grouped = dev_notes.groupby('hadm_id') # get groups of encounter ids
for name, group in grouped:
  final_df = group.copy().reset_index(drop=True) # make a copy of notes for that encounter
  for col in cols:
    final_df[col] = np.nan # set the values to nan

  idx = 0 # index to track the final row in the given encounter
  for i, note_row in final_df.iterrows():
    ne = note_row['ne_charttime']
    sub = dev_str.loc[(dev_str['hadm_id'] == name)].reset_index(drop=True) # get the df corresponding to the ecounter
    for j, str_row in sub.iterrows():
      ce = str_row['ce_charttime']
      if ne < ce: # if the variable charttime < note charttime
        idx += 1
        
        # grab the previous values for the variables and break
        for col in cols:
          final_df.iloc[i, final_df.columns.get_loc(col)] = sub.iloc[j-1][col]          
        break               

  # get the last value in the df for the variables
  for col in cols:
    final_df.iloc[idx, final_df.columns.get_loc(col)] = sub.iloc[-1][col]
  
  final_dfs.append(final_df) # append the df to the list

# cat the list to get final df and reset index
final_df = pd.concat(final_dfs)
final_df.reset_index(inplace=True, drop=True)

In [ ]:
move = ['ne_charttime', 'note']
order = (final_df.columns.drop(move).tolist()) + move 
final_df = final_df[order]

In [ ]:
final_df.loc[(final_df['hadm_id'] == 196673)]

## Question

In [ ]:
ds = {
  'hadm_id': [140694, 140694, 140694, 171544, 171544, 171544, 171544, 171544,],
  'charttime': [pd.to_datetime('2121-08-12 19:00:00'), pd.to_datetime('2121-08-12 19:45:00'), pd.to_datetime('2121-08-12 20:00:00'), pd.to_datetime('2153-09-06 14:11:00'), pd.to_datetime('2153-09-06 17:30:00'), pd.to_datetime('2153-09-06 17:35:00'), pd.to_datetime('2153-09-06 17:40:00'), pd.to_datetime('2153-09-06 17:45:00')],
  'hr': [67.0, 68.0, 70.0, 80.0, 80.0, 80.0, 76.0, 79.0],
  'sbp': [102.0, 135.0, 153.0, 114.0, 114.0, 114.0, 115.0, 117.0],
  'dbp': [75.0, 68.0, 94.0, 50.0, 50.0, 50.0, 51.0, 53.0],
}

dn = {
  'hadm_id': [140694, 140694, 140694, 171544, 171544, 171544, 171544, 171544,],
  'charttime': [pd.to_datetime('2121-08-10 20:32:00'), pd.to_datetime('2121-08-11 12:57:00'), pd.to_datetime('2121-08-11 15:18:00'), pd.to_datetime('2153-09-05 15:09:00'), pd.to_datetime('2153-09-05 17:43:00'), pd.to_datetime('2153-09-06 10:36:00'), pd.to_datetime('2153-09-06 15:55:00'), pd.to_datetime('2153-09-06 17:12:00')],
  'note': ['some text1', 'some text2', 'some text3', 'some text4', 'some text5', 'some text6', 'some text7', 'some text8']
}

In [ ]:
ds = pd.DataFrame(ds)
dn = pd.DataFrame(dn)

In [ ]:
ds

In [ ]:
dn

In [ ]:
final = {
  'hadm_id': [140694, 140694, 140694, 140694, 140694, 140694, 171544, 171544, 171544, 171544, 171544, 171544, 171544, 171544, 171544, 171544],
  'charttime': [pd.to_datetime('2121-08-10 20:32:00'), pd.to_datetime('2121-08-11 12:57:00'), pd.to_datetime('2121-08-11 15:18:00'), pd.to_datetime('2121-08-12 19:00:00'), pd.to_datetime('2121-08-12 19:45:00'), pd.to_datetime('2121-08-12 20:00:00'), pd.to_datetime('2153-09-05 15:09:00'), pd.to_datetime('2153-09-05 17:43:00'), pd.to_datetime('2153-09-06 10:36:00'), pd.to_datetime('2153-09-06 14:11:00'), pd.to_datetime('2153-09-06 15:55:00'), pd.to_datetime('2153-09-06 17:12:00'), pd.to_datetime('2153-09-06 17:30:00'), pd.to_datetime('2153-09-06 17:35:00'), pd.to_datetime('2153-09-06 17:40:00'), pd.to_datetime('2153-09-06 17:45:00')],
  'ce_charttime': [pd.NaT, pd.NaT, pd.NaT, pd.to_datetime('2121-08-12 19:00:00'), pd.to_datetime('2121-08-12 19:45:00'), pd.to_datetime('2121-08-12 20:00:00'), pd.NaT, pd.NaT, pd.NaT, pd.to_datetime('2153-09-06 14:11:00'), pd.NaT, pd.NaT, pd.to_datetime('2153-09-06 17:30:00'), pd.to_datetime('2153-09-06 17:35:00'), pd.to_datetime('2153-09-06 17:40:00'), pd.to_datetime('2153-09-06 17:45:00')],
  'hr': [np.nan, np.nan, np.nan, 67.0, 68.0, 70.0, np.nan, np.nan, np.nan, 80.0, np.nan, np.nan, 80.0, 80.0, 76.0, 76.0],
  'sbp': [np.nan, np.nan, np.nan, 102.0, 135.0, 153.0, np.nan, np.nan, np.nan, 114.0, np.nan, np.nan, 114.0, 114.0, 115.0, 117.0],
  'dbp': [np.nan, np.nan, np.nan, 75.0, 68.0, 94.0, np.nan, np.nan, np.nan, 50.0, np.nan, np.nan, 50.0, 50.0, 51.0, 53.0],
  'ne_charttime': [pd.to_datetime('2121-08-10 20:32:00'), pd.to_datetime('2121-08-11 12:57:00'), pd.to_datetime('2121-08-11 15:18:00'), pd.NaT, pd.NaT, pd.NaT, pd.to_datetime('2153-09-05 15:09:00'), pd.to_datetime('2153-09-05 17:43:00'), pd.to_datetime('2153-09-06 10:36:00'), pd.NaT, pd.to_datetime('2153-09-06 15:55:00'), pd.to_datetime('2153-09-06 17:12:00'), pd.NaT, pd.NaT, pd.NaT, pd.NaT],
  'note': ['some text1', 'some text2', 'some text3', np.nan, np.nan, np.nan, 'some text4', 'some text5', 'some text6', np.nan, 'some text7', 'some text8', np.nan, np.nan, np.nan, np.nan] 
}